# A notebook to find the lowest energy conformer of a TS

This notebook will be similar to the `molecule_torsion` notebook but will require more setup to get the a rough TS geometry.

The step we plan to do are as follows:

1) Create a reasonable TS geometry using the existing framework provided in AutoTST

2) Add pseudo bonds the `reactants` molecule where the `1*`, `2*`, and `3*` atoms are

3) Identify all possible torsions

4) Run through all the possible conformers and calculate the lowest energy conformer from RDKit

For step for, we will do a systematic operation of adding the pseudo bonds to the molecule, setting the torsions, deleting the pseudo bonds, and calculating the energy. This will be done for each conformer and _ideally_ the lowest energy conformer would be further optimized using a quantum package

#### NOTE: We may also need to look at the angle created by `1*`, `2*`, and `3*` ~ This will be studied once the torsional part is up and running

In [1]:
import os
import sys
import cPickle as pkl
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import re
import imp
import itertools
import random
import numpy as np
from numpy import array
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns


# do this before we have a chance to import openbabel!
import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from rdkit.Chem.rdMolTransforms import *

import py3Dmol

from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.kinetics import PDepArrhenius, PDepKineticsModel

from rmgpy.data.rmg import RMGDatabase
from rmgpy.data.kinetics import KineticsDepository, KineticsRules
from rmgpy.qm.main import QMCalculator, QMSettings
from rmgpy.qm.qmdata import QMData
from rmgpy.qm.reaction import QMReaction
from rmgpy.qm.molecule import QMMolecule


/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
rxnFamily = ['H_Abstraction']
print('Loading RMG Database ...')
rmgDatabase = RMGDatabase()
databasePath = os.path.abspath(os.path.join(os.getenv('RMGpy', '..'), '..', 'RMG-database', 'input'))
print(databasePath)
rmgDatabase.load(databasePath,
                 kineticsFamilies=rxnFamily,
                 transportLibraries=[],
                 reactionLibraries=[],
                 seedMechanisms=[],
                 thermoLibraries=['primaryThermoLibrary', 'KlippensteinH2O2', 'thermo_DFT_CCSDTF12_BAC', 'CBS_QB3_1dHR' ],
                 solvation=False,
                 )
print('RMG Database Loaded')


## We are using a ts Pickle from discovery. 
## I was running into issues performing ts opts locally, so this is a workaround
f = open("../tsDatabase.pkl", "r")
tsDatabase = pkl.load(f)

settings = QMSettings(
    software='gaussian',
    method='m062x',
    fileStore=os.path.expandvars('.'),
    scratchDirectory=os.path.expandvars('.'),
    )

thermo.py:760 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:774 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...


Loading RMG Database ...
/Users/nathan/Code/RMG-database/input


transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
database.py:165 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-database/input/kinetics/families
statmech.py:526 loadGroups INFO Loading frequencies group database from /Users/nathan/Code/RMG-database/input/statmech/groups...


RMG Database Loaded


In [3]:
def view_mol(mol):
    """
    A function designed to visulaize rdkit molecules
    
    Input: 
    * mol (an rdkit molecule object)
    
    Output: 
    * 3D figure of the molecule from py3Dmol
    
    """
    mb  = Chem.MolToMolBlock(mol)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb, "sdf")
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p.show()

In [4]:
test_mol = Chem.AddHs(Chem.MolFromSmiles(
    "COC(=O)Cc1ccccc1CC2CC(O)C(C(=C)CC#CCC=O)C2OO"
))

AllChem.EmbedMolecule(test_mol)

view_mol(test_mol)

In [51]:
def calc_energy(mol):
    """
    A function designed to calculate the energy of a molecule or TS.
    
    Input: 
    * rdkit molecule object
    
    Output: 
    * energy of the geometry calculated using rdkit's forcefield method
    * the resultant rdkit molecule object
    """
    # Setting the force field parameters
    """try:
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(mol)
    except:
        # Picking an arbitrarly large molecule to create a dummy mol properties forcefield object
        # This is primarily used for TSs
        test_mol = Chem.AddHs(Chem.MolFromSmiles(
            "COC(=O)Cc1ccccc1CC2CC(O)C(C(=C)CC#CCC=O)C2OO"
        ))
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(test_mol)"""
        
    ff = rdkit.Chem.ChemicalForceFields.UFFGetMoleculeForceField(mol)#, molprop)
    
    
    
    return ff.CalcEnergy(), mol

In [52]:
def constrained_energy_calc(mol, list_of_torsions, angles):
    """
    A function designed to calculate the energy of a molecule or TS.
    For this function, the molecule torsions are fixed, 
    but the rest of the molecule is allowed to relax to a local minimum
    
    Input: 
    * rdkit molecule object
    * list of torsions as (i,j,k,l) tuples
    * list of angles to set the corresponding torsions to
    
    Output: 
    * energy of the geometry calculated using rdkit's forcefield method
    * the resultant rdkit molecule object
    """
    
    # Setting the force field parameters
    """try:
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(mol)
    except:
    
        # Picking an arbitrarly large molecule to create a dummy mol properties forcefield object
        # This is primarily used for TSs
        test_mol = Chem.AddHs(Chem.MolFromSmiles(
            "COC(=O)Cc1ccccc1CC2CC(O)C(C(=C)CC#CCC=O)C2OO"
        ))
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(test_mol)"""
    ff = rdkit.Chem.ChemicalForceFields.UFFGetMoleculeForceField(mol)#, molprop)
    
    # Zipping the torsion list and the angles together
    geometries = zip(list_of_torsions, angles)
    
    # Generating a conformer to edit
    tup = mol.GetConformers()
    conformer = tup[0]
    
    # Setting the corresponding torsion to their proper angles
    for geometry in geometries:
        
        i = geometry[0][0]
        j = geometry[0][1]
        k = geometry[0][2]
        l = geometry[0][3]
        angle = geometry[1]
        
        SetDihedralDeg(conformer,
                           i, 
                           j, 
                           k, 
                           l,
                           angle)
        
        # Constraining the torsions of the molecule
        ff.UFFAddTorsionConstraint(i,j,k,l, True, angle, angle, 1)
     

    #e, m = calc_energy(mol)
    # Optimizing the molecule
    AllChem.UFFOptimizeMolecule(mol)
    
    
    
    return ff.CalcEnergy(), mol

In [53]:
def optimize_and_calc(mol, list_of_torsions, angles):
    """
    A function designed to calculate the energy of a molecule or TS.
    For this function, the torsions of the molecule are set to their corresponding angles,
    then the molecule is allowed to relax to a local minimum
    
    Input: 
    * rdkit molecule object
    * list of torsions as (i,j,k,l) tuples
    * list of angles to set the corresponding torsions to
    
    Output: 
    * energy of the geometry calculated using rdkit's forcefield method
    * the resultant rdkit molecule object
    """
    
    # Setting the force field parameters
    """try:
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(mol)
    except:
        # Picking an arbitrarly large molecule to create a dummy mol properties forcefield object
        # This is primarily used for TSs
        test_mol = Chem.AddHs(Chem.MolFromSmiles(
            "COC(=O)Cc1ccccc1CC2CC(O)C(C(=C)CC#CCC=O)C2OO"
        ))
        
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(test_mol) """
    ff = rdkit.Chem.ChemicalForceFields.UFFGetMoleculeForceField(mol)# molprop)
    
    # Zipping the torsions and their corresponding angles together
    geometries = zip(list_of_torsions, angles)
    
    # Generating conformers and selecting one
    tup = mol.GetConformers()
    conformer = tup[0]
    
    for geometry in geometries:
        # Setting the torsion angles
        i = geometry[0][0]
        j = geometry[0][1]
        k = geometry[0][2]
        l = geometry[0][3]
        angle = geometry[1]
        
        SetDihedralDeg(conformer,
                           i, 
                           j, 
                           k, 
                           l,
                           angle)
    
    # Allowing the molecule to relax
    AllChem.UFFOptimizeMolecule(mol)
    
    
    return ff.CalcEnergy(), mol

In [8]:
def get_torsion_list(RDMol):
    """
    A function to return a list of the possible torsions in an RDKit molecule
    
    Input:
    * RDKit molecule 
    
    Output:
    * list of tuples containing 4 indicies of atoms representing a torsion
    """
    torsion_list = []
    for bond1 in RDMol.GetBonds():
        atom1 = bond1.GetBeginAtom()
        atom2 = bond1.GetEndAtom()
        if atom1.IsInRing() or atom2.IsInRing():
            # Making sure that bond1 we're looking at are in a ring
            continue

        bond_list1 = list(atom1.GetBonds())
        bond_list2 = list(atom2.GetBonds())

        if not len(bond_list1) > 1 and not len(bond_list2) > 1:
            # Making sure that there are more than one bond attached to
            # the atoms we're looking at
            continue
            
        # Getting the 0th and 3rd atom and insuring that atoms 
        # attached to the 1st and 2nd atom are not terminal hydrogens
        # We also make sure that all of the atoms are properly bound together

        # If the above are satisified, we append a tuple of the torsion our torsion_list
        got_atom0 = False
        got_atom3 = False
            
        for bond0 in bond_list1:
            atomX = bond0.GetOtherAtom(atom1)
            if atomX.GetAtomicNum() == 1 and len(atomX.GetBonds()) == 1:
                # This means that we have a terminal hydrogen, skip this
                # NOTE: for H_abstraction TSs, a non teminal H should exist
                continue 
            if atomX.GetIdx() != atom2.GetIdx():
                got_atom0 = True
                atom0 = atomX

        for bond2 in bond_list2:
            atomY = bond2.GetOtherAtom(atom2)
            if atomY.GetAtomicNum() == 1 and len(atomY.GetBonds()) == 1:
                # This means that we have a terminal hydrogen, skip this
                continue 
            if atomY.GetIdx() != atom1.GetIdx():
                got_atom3 = True
                atom3 = atomY
        
        if not (got_atom0 and got_atom3):
            # Making sure atom0 and atom3 were not found
            continue

        # Looking to make sure that all of the atoms are properly bonded to eached
        if (
            RDMol.GetBondBetweenAtoms(atom0.GetIdx(), atom1.GetIdx()) and 
            RDMol.GetBondBetweenAtoms(atom1.GetIdx(), atom2.GetIdx()) and 
            RDMol.GetBondBetweenAtoms(atom2.GetIdx(), atom3.GetIdx())   ) : 

            torsion_tup = (atom0.GetIdx(), atom1.GetIdx(), atom2.GetIdx(), atom3.GetIdx())
            torsion_list.append(torsion_tup)
        
    return torsion_list

# Creating a reaction to test

### Making a normal RMG reaction

In [9]:
rSpecies1 = Molecule(SMILES="CCCCCC")
rSpecies2 = Molecule(SMILES="[O]O")
pSpecies2 = Molecule(SMILES="[CH2]CCCCC")
pSpecies1 = Molecule(SMILES="OO")


input_reaction = Reaction(reactants=[rSpecies1, rSpecies2], products=[pSpecies1, pSpecies2], reversible=True)
input_reaction

### Generating a list of reactions from our imported reaction

In [22]:
rxnFamilies = ["H_Abstraction"]
rSpecies1, rSpecies2 = input_reaction.reactants
pSpecies1, pSpecies2 = input_reaction.products

for species in itertools.chain(input_reaction.reactants, input_reaction.products):
        species = species.generateResonanceIsomers()

testReaction = Reaction(
    reactants = input_reaction.reactants, 
    products = input_reaction.products, 
    reversible = True)

reactants = [species for species in input_reaction.reactants]
products = [species for species in input_reaction.products]

reactionList = []

checkRxn = rmgDatabase.kinetics.generateReactionsFromFamilies(
    reactants, 
    products, 
    only_families=rxnFamilies)

for rxn0 in checkRxn:
    reactionList.append(rxn0)
logging.info("generateReactionsFromFamilies successfuly performed")

reactionList


<ipython-input-22-532086f4ac92>:25 <module> INFO generateReactionsFromFamilies successfuly performed


[Reaction(reactants=[Molecule(SMILES="CCCCCC"), Molecule(SMILES="[O]O")], products=[Molecule(SMILES="OO"), Molecule(SMILES="[CH2]CCCCC")], degeneracy=6, pairs=[[Molecule(SMILES="CCCCCC"), Molecule(SMILES="[CH2]CCCCC")], [Molecule(SMILES="[O]O"), Molecule(SMILES="OO")]])]

### Labeling the atoms in the reaction

In [25]:
gotOne=False
for reaction in reactionList:
    # Check if any of the RMG proposed reactions matches the reaction in the mechanism
    if testReaction.isIsomorphic(reaction):
        print "Found matching reaction"
        # Now add the labeled atoms to the Molecule, and check all labels were added
        atLblsR = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])
        atLblsP = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])

        for reactant in reaction.reactants:
            reactant.clearLabeledAtoms()
            for atom in reactant.atoms:
                for atomLabel in reaction.labeledAtoms:
                    if atom==atomLabel[1]:
                        atom.label = atomLabel[0]
                        atLblsR[atomLabel[0]] = True
                        
        for product in reaction.products:
            product.clearLabeledAtoms()
            for atom in product.atoms:
                for atomLabel in reaction.labeledAtoms:
                    if atom==atomLabel[1]:
                        atom.label = atomLabel[0]
                        atLblsP[atomLabel[0]] = True
                        
        if all( atLblsR.values() ) and all( atLblsP.values() ):
            # We successfully labeled all of the atoms
            gotOne=True 
            break
            
rxn = QMReaction(reaction=reaction, settings=settings, tsDatabase=tsDatabase)
rxn.reaction.family

Found matching reaction


'H_Abstraction'

## Making a single RDKit molecule from the RMG QM reaction

In [26]:
reactant, product = rxn.setupMolecules()
mol = reactant.toRDKitMol(removeHs=False)
AllChem.EmbedMolecule(mol)
view_mol(mol)

## Creating a reasionable goemetry

In [34]:
labels, atomMatch = rxn.getLabels(reactant)

tsRDMol, bm, rxn.reactantGeom = rxn.generateBoundsMatrix(reactant)

bm = rxn.editMatrix(reactant, bm, labels)

tsRDMol = rxn.reactantGeom.rd_embed(tsRDMol, 15, bm=bm, match=atomMatch)[0]

tsRDMol = rdkit.Chem.rdchem.RWMol(tsRDMol)

view_mol(tsRDMol)

reaction.py:300 generateBoundsMatrix INFO Generating bounds matrix for CCCCCC.[O]O
reaction.py:338 bmPreEdit INFO Changing lower limit 2.9 to 2.393074
reaction.py:338 bmPreEdit INFO Changing lower limit 3.65 to 3.26214059416
reaction.py:338 bmPreEdit INFO Changing lower limit 2.520343 to 2.457475


## Checking that the atoming labeling matches what was given from AutoTST

In [29]:
tsRDMol_copy = tsRDMol.__copy__() # making a copy of the TS so we don't need to kill the kernal over and over again
# This is because of the above bug
view_mol(tsRDMol_copy)

#### Getting the `1*`, `2*`, and `3*` atoms

In [35]:
for atom in tsRDMol_copy.GetAtoms():
    idx = atom.GetIdx()
    num = atom.GetAtomicNum()
    rmg_atom = reactant.atoms[idx]
    
    if rmg_atom.label:
        if rmg_atom.label == "*1":
            atom1_star = atom
        if rmg_atom.label == "*2":
            atom2_star = atom
        if rmg_atom.label == "*3":
            atom3_star = atom

print atom1_star.GetAtomicNum(), atom2_star.GetAtomicNum(), atom3_star.GetAtomicNum()

    

6 1 8


#### Adding the pseduo bond between either `1*` and `2*` or `2*` and `3*` depending on which bond already exists

In [36]:
bond_between_23 = False
try:
    tsRDMol_copy.AddBond(atom1_star.GetIdx(), atom2_star.GetIdx(), order=rdkit.Chem.rdchem.BondType.SINGLE)
except RuntimeError:
    print "Bond already exists betwee 1* and 2*"
    bond_between_23 = True
    tsRDMol_copy.AddBond(atom2_star.GetIdx(), atom3_star.GetIdx(), order=rdkit.Chem.rdchem.BondType.SINGLE)

    
view_mol(tsRDMol_copy)

Bond already exists betwee 1* and 2*


### Getting the list of torsions for the TS

In [37]:
torsion_list = []
for bond1 in tsRDMol_copy.GetBonds():
    atom1 = bond1.GetBeginAtom()
    atom2 = bond1.GetEndAtom()
    if atom1.IsInRing() or atom2.IsInRing():# atom1.GetAtomicNum() == 1 or atom2.GetAtomicNum() == 1:
        # Making sure that the atoms we're looking at are in a ring
        continue

    bond_list1 = list(atom1.GetBonds())
    bond_list2 = list(atom2.GetBonds())

    if not len(bond_list1) > 1 and not len(bond_list2) > 1:
        # Making sure that there are more than one bond attached to
        # the atoms we're looking at
        continue


    # Getting the 0th and 3rd atom and insuring that atoms 
    # attached to the 1st and 2nd atom are not terminal hydrogens
    # We also make sure that all of the atoms are properly bound together
    
    # If the above are satisified, we append a tuple of the torsion our torsion_list

    for bond0 in bond_list1:
        atomX = bond0.GetOtherAtom(atom1)
        if atomX.GetAtomicNum() == 1 and len(atomX.GetBonds()) == 1:
            # This means that we have a terminal hydrogen, skip this
            # NOTE: for H_abstraction TSs, a non teminal H should exist
            continue 
        if atomX.GetIdx() != atom2.GetIdx():
            atom0 = atomX

    for bond2 in bond_list2:
        atomY = bond2.GetOtherAtom(atom2)
        if atomY.GetAtomicNum() == 1 and len(atomY.GetBonds()) == 1:
            # This means that we have a terminal hydrogen, skip this
            continue 
        if atomY.GetIdx() != atom1.GetIdx():
            atom3 = atomY
        
    # Looking to make sure that all of the atoms are properly bonded to eached
    if (
        tsRDMol_copy.GetBondBetweenAtoms(atom0.GetIdx(), atom1.GetIdx()) and 
        tsRDMol_copy.GetBondBetweenAtoms(atom1.GetIdx(), atom2.GetIdx()) and 
        tsRDMol_copy.GetBondBetweenAtoms(atom2.GetIdx(), atom3.GetIdx())   ) : 
        
        torsion_tup = (atom0.GetIdx(), atom1.GetIdx(), atom2.GetIdx(), atom3.GetIdx())
        torsion_list.append(torsion_tup)

torsion_angles = np.arange(0, 360+30,30)
torsion_combos = list( itertools.combinations_with_replacement( torsion_angles, len(torsion_list)) )
torsion_combos = torsion_combos + list(itertools.combinations_with_replacement( torsion_angles[::-1], len(torsion_list)) )

torsion_list

[(2, 0, 1, 3),
 (1, 0, 2, 4),
 (0, 1, 3, 5),
 (0, 2, 4, 14),
 (2, 4, 14, 21),
 (4, 14, 21, 20)]

##### Testing to see if the above script can be performed using `get_torsion_list` that we made from `molecule_torsion`

In [38]:
get_torsion_list(tsRDMol_copy)

[(2, 0, 1, 3),
 (1, 0, 2, 4),
 (0, 1, 3, 5),
 (0, 2, 4, 14),
 (2, 4, 14, 21),
 (4, 14, 21, 20)]

#### Setting the torsions for a single conformer. 

From this point, we are working with `tsRDMol_store`. This means that we can edit this copy and re-copy `tsRDMol` rather than have to completely recreate the geometry

In [39]:
# Making a copy of the TS molecule and etting a conformer to edit
tup = tsRDMol_copy.GetConformers()
conformer = tup[0]


# Picking a random torsion combo
geometry = zip(torsion_list, torsion_combos[2000])

for torsion in geometry:
    print torsion
    i = torsion[0][0]
    j = torsion[0][1]
    k = torsion[0][2]
    l = torsion[0][3]
    angle = torsion[1]

    SetDihedralDeg(conformer,
                   i, 
                   j, 
                   k, 
                   l,
                   angle)


view_mol(tsRDMol_copy)

((2, 0, 1, 3), 0)
((1, 0, 2, 4), 30)
((0, 1, 3, 5), 30)
((0, 2, 4, 14), 90)
((2, 4, 14, 21), 210)
((4, 14, 21, 20), 270)


#### Removing the pseudo bond from the copy molecule and calculating energy

**TODO:** Possibly use UFF instead of MMFF? Read the papers to know what the differences are

In [68]:
if bond_between_23 == False:
    tsRDMol_copy.RemoveBond(atom1_star.GetIdx(), atom2_star.GetIdx())
else:
    tsRDMol_copy.RemoveBond(atom2_star.GetIdx(), atom3_star.GetIdx())

print calc_energy(tsRDMol_copy)
view_mol(tsRDMol_copy)

(502670.1982614912, <rdkit.Chem.rdchem.RWMol object at 0x1152bcfa0>)


Yay!!! We got this working for a single TS! Now let's look at an example with just a single torsion and visualize a plot of the torsion.

# Creating functions to do the above automatically

We will use the same `input_reaction` for this

**Note** We're on to something, but we have some issues with optimization... We need to fix the distances between the `1*`, `2*`, and `3*` atoms

In [45]:
rSpecies1 = Molecule(SMILES="CCCCCC")
rSpecies2 = Molecule(SMILES="[O]O")
pSpecies2 = Molecule(SMILES="[CH2]CCCCC")
pSpecies1 = Molecule(SMILES="OO")


input_reaction = Reaction(reactants=[rSpecies1, rSpecies2], products=[pSpecies1, pSpecies2], reversible=True)
input_reaction

In [47]:
def get_ts_rdmol(input_reaction):
    """

    """
    rxnFamilies = ["H_Abstraction"]
    rSpecies1, rSpecies2 = input_reaction.reactants
    pSpecies1, pSpecies2 = input_reaction.products

    for species in itertools.chain(input_reaction.reactants, input_reaction.products):
            species = species.generateResonanceIsomers()

    testReaction = Reaction(
        reactants = input_reaction.reactants,
        products = input_reaction.products,
        reversible = True)

    reactants = [species for species in input_reaction.reactants]
    products = [species for species in input_reaction.products]

    reactionList = []

    checkRxn = rmgDatabase.kinetics.generateReactionsFromFamilies(
        reactants,
        products,
        only_families=rxnFamilies)

    for rxn0 in checkRxn:
        reactionList.append(rxn0)
    logging.info("generateReactionsFromFamilies successfuly performed")

    gotOne=False
    for reaction in reactionList:
        # Check if any of the RMG proposed reactions matches the reaction in the mechanism
        if testReaction.isIsomorphic(reaction):
            #print "Found matching reaction"
            # Now add the labeled atoms to the Molecule, and check all labels were added
            atLblsR = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])
            atLblsP = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])

            for reactant in reaction.reactants:
                reactant.clearLabeledAtoms()
                for atom in reactant.atoms:
                    for atomLabel in reaction.labeledAtoms:
                        if atom==atomLabel[1]:
                            atom.label = atomLabel[0]
                            atLblsR[atomLabel[0]] = True

            for product in reaction.products:
                product.clearLabeledAtoms()
                for atom in product.atoms:
                    for atomLabel in reaction.labeledAtoms:
                        if atom==atomLabel[1]:
                            atom.label = atomLabel[0]
                            atLblsP[atomLabel[0]] = True

            if all( atLblsR.values() ) and all( atLblsP.values() ):
                # We successfully labeled all of the atoms
                gotOne=True
                break

    rxn = QMReaction(reaction=reaction, settings=settings, tsDatabase=tsDatabase)


    reactant, product = rxn.setupMolecules()
    mol = reactant.toRDKitMol(removeHs=False)

    AllChem.EmbedMolecule(mol)
    labels, atomMatch = rxn.getLabels(reactant)
    tsRDMol, bm, rxn.reactantGeom = rxn.generateBoundsMatrix(reactant)
    bm = rxn.editMatrix(reactant, bm, labels)
    tsRDMol = rxn.reactantGeom.rd_embed(tsRDMol, 15, bm=bm, match=atomMatch)[0]
    tsRDMol = rdkit.Chem.rdchem.RWMol(tsRDMol)

    return tsRDMol

tsRDMol = get_ts_rdmol(input_reaction)
view_mol(tsRDMol)

<ipython-input-47-a7e8e2ce1cec>:29 get_ts_rdmol INFO generateReactionsFromFamilies successfuly performed
reaction.py:300 generateBoundsMatrix INFO Generating bounds matrix for CCCCCC.[O]O


Found matching reaction


reaction.py:338 bmPreEdit INFO Changing lower limit 2.9 to 2.393074
reaction.py:338 bmPreEdit INFO Changing lower limit 3.65 to 3.26214059416
reaction.py:338 bmPreEdit INFO Changing lower limit 2.520343 to 2.457475
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 1 of 15


In [48]:
def create_pseudo_geometry(tsRDMol):

    for atom in tsRDMol.GetAtoms():
        idx = atom.GetIdx()
        num = atom.GetAtomicNum()
        rmg_atom = reactant.atoms[idx]

        if rmg_atom.label:
            if rmg_atom.label == "*1":
                atom1_star = atom
            if rmg_atom.label == "*2":
                atom2_star = atom
            if rmg_atom.label == "*3":
                atom3_star = atom

    bond_between_23 = False
    try:
        tsRDMol.AddBond(atom1_star.GetIdx(), atom2_star.GetIdx(), order=rdkit.Chem.rdchem.BondType.SINGLE)
    except RuntimeError:
        #print "Bond already exists betwee 1* and 2*"
        bond_between_23 = True
        tsRDMol.AddBond(atom2_star.GetIdx(), atom3_star.GetIdx(), order=rdkit.Chem.rdchem.BondType.SINGLE)

    return tsRDMol

tsRDMol = create_pseudo_geometry(tsRDMol)
view_mol(tsRDMol)

Bond already exists betwee 1* and 2*


In [63]:
view_mol(tsRDMol)

In [57]:
def get_ts_torsion_energies(tsRDMol, delta):

    torsion_list = get_torsion_list(tsRDMol)

    torsion_combos = list( itertools.combinations_with_replacement(torsion_angles, len(torsion_list)) )
    if len(torsion_list) != 1:
        torsion_combos = list(
            set(
                torsion_combos +
                list(itertools.combinations_with_replacement(
                    torsion_angles[::-1], len(torsion_list)
                ))))
    df = []

    tup = tsRDMol.GetConformers()
    conformer = tup[0]

    for combo in torsion_combos:
        geometry = zip(torsion_list, combo)
        for torsion in geometry:
            #print torsion
            i = torsion[0][0]
            j = torsion[0][1]
            k = torsion[0][2]
            l = torsion[0][3]
            angle = torsion[1]

            SetDihedralDeg(conformer,
                           i,
                           j,
                           k,
                           l,
                           angle)

        if bond_between_23 == False:
            tsRDMol.RemoveBond(atom1_star.GetIdx(), atom2_star.GetIdx())
        else:
            tsRDMol.RemoveBond(atom2_star.GetIdx(), atom3_star.GetIdx())

        e0, m0 = calc_energy(tsRDMol)
        e1, m1 = constrained_energy_calc(tsRDMol, torsion_list, combo)
        e2, m2 = optimize_and_calc(tsRDMol, torsion_list, combo)
            
        df.append( [e0, e1, e2,
                   m0, m1, m2] 
                  + list(combo))


    df = pd.DataFrame(df)
    columns = ["No Optimization Energy", "Partial Optimization Energy", "Full Optimization Energy",
              "No Optimization Mol", "Partial Optimization Mol", "Full Optimization Mol"]
    for i in range(len(torsion_list)):
        columns.append("Torsion " + str(i))

    df.columns = columns

    return df

df = get_ts_torsion_energies(tsRDMol, 60)


,No Optimization Energy,Partial Optimization Energy,Full Optimization Energy,No Optimization Mol,Partial Optimization Mol,Full Optimization Mol,Torsion 0,Torsion 1,Torsion 2
0,7.632473e+12,31.911774,8.428359,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,360,360,30
1,9.026391e+01,8.430458,7.196169,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,300,240,60
2,2.389201e+06,14.822590,7.196169,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,30,30,270
3,6.913594e+04,22.385285,7.267500,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,0,30,180
4,3.195236e+02,13.187252,8.428359,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,330,270,120
5,1.319099e+01,11.685464,6.786869,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,60,120,120
6,1.518535e+01,17.823691,10.198245,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,270,120,120
7,5.023365e+08,9.814428,5.686614,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,60,300,330
8,8.399894e+00,13.492779,4.458614,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,270,180,150
9,1.014983e+03,22.457526,7.196169,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,60,150,360


In [59]:
df.sort("No Optimization Energy")

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,No Optimization Energy,Partial Optimization Energy,Full Optimization Energy,No Optimization Mol,Partial Optimization Mol,Full Optimization Mol,Torsion 0,Torsion 1,Torsion 2
827,4.638655e+00,18.066554,3.262147,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,180,180,180
33,5.694540e+00,16.410605,4.424447,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,180,180,270
171,5.814310e+00,16.720376,4.458614,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,270,180,180
843,5.839933e+00,15.553878,4.458614,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,90,180,180
139,6.068823e+00,15.324997,3.262147,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,210,180,180
759,6.547218e+00,15.324997,3.262147,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,180,180,210
538,6.559400e+00,15.324997,3.262147,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,150,180,180
241,6.979880e+00,15.863326,4.424447,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,180,180,90
859,7.039689e+00,8.831730,5.474753,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,210,300,300
536,7.101962e+00,11.859594,4.458614,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,<rdkit.Chem.rdchem.RWMol object at 0x117169c58>,60,150,180


In [58]:
def get_lowest_ts_energy_conformers(tsRDMol, delta):
    """
    A function that can return 
    
    """
    
    conformer_dict = {}
    
    df = get_ts_torsion_energies(tsRDMol, delta)
    
    conformer_dict["No Optimization"] = (
        df.sort("No Optimization Energy")["No Optimization Energy"].iloc[0],
        df.sort("No Optimization Energy")["No Optimization Mol"].iloc[0]
    )
    
    conformer_dict["Partial Optimization"] = (
        df.sort("Partial Optimization Energy")["Partial Optimization Energy"].iloc[0],
        df.sort("Partial Optimization Energy")["Partial Optimization Mol"].iloc[0]
    )
    
    conformer_dict["Full Optimization"] = (
        df.sort("Full Optimization Energy")["Full Optimization Energy"].iloc[0],
        df.sort("Full Optimization Energy")["Full Optimization Mol"].iloc[0]
    )
    
    return conformer_dict

d = get_lowest_ts_energy_conformers(tsRDMol, 60)
d

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


{'Full Optimization': (3.2621470149789271,
  <rdkit.Chem.rdchem.RWMol at 0x117169c58>),
 'No Optimization': (4.6386563133716798,
  <rdkit.Chem.rdchem.RWMol at 0x117169c58>),
 'Partial Optimization': (4.9070809360510825,
  <rdkit.Chem.rdchem.RWMol at 0x117169c58>)}

In [62]:
view_mol(d["No Optimization"][1])

# Performing the above for a single reaction 

# The reaction of interest is $CH_4 + OH = CH_3 + H_2O$

#### The goal of the following function is to try and combine three different parts of the above script. This should create a test reaction, label the atoms, and set the reaction family... I still think parts of this are redundant... but it works... 

In [ ]:


rSpecies1 = Molecule(SMILES="CCC(C)(C)O")
rSpecies2 = Molecule(SMILES="[O]O")
pSpecies2 = Molecule(SMILES="[CH2]CC(C)(C)O")
pSpecies1 = Molecule(SMILES="OO")


single_rotor_rxn = Reaction(reactants=[rSpecies1, rSpecies2], products=[pSpecies1, pSpecies2], reversible=True)
single_rotor_rxn

In [ ]:


def all_in_one(oohReaction):
    rxnFamily = "H_Abstraction"
    rSpecies1, rSpecies2 = oohReaction.reactants
    pSpecies1, pSpecies2 = oohReaction.products

    rSpecies1.generateResonanceIsomers()
    rSpecies2.generateResonanceIsomers()
    pSpecies1.generateResonanceIsomers()
    pSpecies2.generateResonanceIsomers()

    testReaction = Reaction(reactants=[rSpecies1, rSpecies2], products=[pSpecies1, pSpecies2], reversible=True)
    reactionList = []
    tempList = rmgDatabase.kinetics.generateReactionsFromFamilies([rSpecies1, rSpecies2], [], only_families=[rxnFamily])
    for rxn0 in tempList:
        reactionList.append(rxn0)
    logging.info("generateReactionsFromFamilies successfuly performed")
    #print reactionList
    
    
    gotOne=False
    for reaction in reactionList:
        # Check if any of the RMG proposed reactions matches the reaction in the mechanism
        if testReaction.isIsomorphic(reaction):
            print "Found matching reaction"
            # Now add the labeled atoms to the Molecule, and check all labels were added
            atLblsR = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])
            atLblsP = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])

            for reactant in reaction.reactants:

                reactant.clearLabeledAtoms()
                for atom in reactant.atoms:
                    for atomLabel in reaction.labeledAtoms:
                        if atom==atomLabel[1]:
                            atom.label = atomLabel[0]
                            atLblsR[atomLabel[0]] = True
            for product in reaction.products:

                product.clearLabeledAtoms()
                for atom in product.atoms:
                    for atomLabel in reaction.labeledAtoms:
                        if atom==atomLabel[1]:
                            atom.label = atomLabel[0]
                            atLblsP[atomLabel[0]] = True
            if all( atLblsR.values() ) and all( atLblsP.values() ):
                gotOne=True
                break
    chemkinRxn = testReaction
    
    rxnFamily = "H_Abstraction"
    rxnFamilies = ['H_Abstraction']
    logging.info("chemkinRxn: {!r}".format(chemkinRxn))
    # Ensure all resonance isomers have been generated
    logging.info('Ensuring resonance isomers have been generated')
    for species in itertools.chain(chemkinRxn.reactants, chemkinRxn.products):
        print(type(species))
        species = species.generateResonanceIsomers()

    testReaction = Reaction(reactants=chemkinRxn.reactants, products=chemkinRxn.products, reversible=True)

    reactants = [species for species in chemkinRxn.reactants]
    # reactant_molecules is a list of lists of resonance isomers,
    # eg. a bimolecular reaction where the second reactant has 2 isomers is: [[r1],[r2i1,r2i2]]

    products = [species for species in chemkinRxn.products]
    # products is a list of molecule objects (only one resonance form of each product), eg [p1, p2]
    print reactants
    #print type(reactant_molecules[0])
    print products

    checkRxn = rmgDatabase.kinetics.generateReactionsFromFamilies(reactants, products, only_families=rxnFamilies)# reversible=True)
    if len(checkRxn) == 1:
        logging.info("Generated one reaction:")
        logging.info(checkRxn)
        reaction = checkRxn[0]
    else:  # didn't break from for loop
        for reaction in checkRxn:
            logging.info("Generated these reactions:")
            logging.info(reaction)
            reaction = checkRxn[0]
        raise Exception("Couldn't generate exactly one reaction matching {} in family {}".format(chemkinRxn, rxnFamilies))

    logging.info("The reaction of interest is as follows: ")
    logging.info(reaction)

    logging.info("asserting that the testReaction is Isomorphic")
    assert testReaction.isIsomorphic(reaction)
    logging.info("reaction: {!r}".format(reaction))

    atLblsR = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])
    atLblsP = dict([(lbl[0], False) for lbl in reaction.labeledAtoms])

    gotOne = False
    logging.info("Labeling reactant atoms")
    for reactant in reaction.reactants:
        reactant = reactant
        reactant.clearLabeledAtoms()
        for atom in reactant.atoms:
            for atomLabel in reaction.labeledAtoms:
                if atom == atomLabel[1]:
                    atom.label = atomLabel[0]
                    atLblsR[atomLabel[0]] = True

    logging.info("Labeling product atoms")
    for product in reaction.products:
        product = product
        product.clearLabeledAtoms()
        for atom in product.atoms:
            for atomLabel in reaction.labeledAtoms:
                if atom == atomLabel[1]:
                    atom.label = atomLabel[0]
                    atLblsP[atomLabel[0]] = True

    if all(atLblsR.values()) and all(atLblsP.values()):
        gotOne = True

    rxnFamily = reaction.family
    return reaction

single_rotor_rxn = all_in_one(single_rotor_rxn)
single_rotor_rxn = QMReaction(reaction=single_rotor_rxn, settings=settings, tsDatabase=tsDatabase)
single_rotor_rxn

In [ ]:

reactant, product = single_rotor_rxn.setupMolecules()
mol = reactant.toRDKitMol(removeHs=False)
AllChem.EmbedMolecule(mol)
view_mol(mol)

### Currently running into issues with the following chunk of code... Also, the issue that I had above isn't there any more... I'm confused... I look into this after lunch

In [ ]:
labels, atomMatch = single_rotor_rxn.getLabels(reactant)

tsRDMol_1, bm, single_rotor_rxn.reactantGeom = single_rotor_rxn.generateBoundsMatrix(reactant)

bm = single_rotor_rxn.editMatrix(reactant, bm, labels)

tsRDMol_1 = single_rotor_rxn.reactantGeom.rd_embed(tsRDMol_1, 15, bm=bm, match=atomMatch)[0]

tsRDMol_1 = rdkit.Chem.rdchem.RWMol(tsRDMol_1)

view_mol(tsRDMol_1)



In [ ]:
def calc_energy_UFF(mol):
    """
    A function designed to calculate the energy of a molecule or TS
    
    Input: rdkit molecule object
    
    Output: energy of the geometry calculated using rdkit's forcefield method
    """
    """try:
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(mol)
    except:
    
        # Picking an arbitrarly large molecule to create a dummy mol properties forcefield object
        test_mol = Chem.AddHs(Chem.MolFromSmiles(
            "COC(=O)Cc1ccccc1CC2CC(O)C(C(=C)CC#CCC=O)C2OO"
        ))
        
        molprop = rdkit.Chem.ChemicalForceFields.MMFFGetMoleculeProperties(test_mol)"""
    ff = rdkit.Chem.ChemicalForceFields.UFFGetMoleculeForceField(mol)
    return ff.CalcEnergy()

In [ ]:
calc_energy_UFF(tsRDMol_store)

In [ ]:
view_mol(tsRDMol_store)